You are evaluating a candidate sentiment model to replace a production baseline. Your goal is to determine whether this model should ship.

“Ship” means: we would choose the candidate model over the baseline for deployment based on the evidence you collect.

### Step 1 - Install the required dependencies, set up W&B and make sure the python version is 3.10 and above

In [1]:
%pip uninstall pyarrow -y
%pip install pyarrow --upgrade

Found existing installation: pyarrow 23.0.0
Uninstalling pyarrow-23.0.0:
  Successfully uninstalled pyarrow-23.0.0
Note: you may need to restart the kernel to use updated packages.
  Using cached pyarrow-23.0.0-cp312-cp312-win_amd64.whl.metadata (3.1 kB)
Using cached pyarrow-23.0.0-cp312-cp312-win_amd64.whl (27.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -q wandb datasets transformers evaluate tqdm emoji regex pandas pyarrow scikit-learn nbformat torch

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import numpy as np
import pandas as pd
import emoji
import wandb

from datasets import load_dataset
from transformers import pipeline

SEED = 42
np.random.seed(SEED)

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os, wandb
wandb.login()

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from C:\Users\STUDENT\_netrc.
wandb: Currently logged in as: sddaiddo (sddaiddo-carnegie-mellon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
!python --version

Python 3.12.4


In [5]:
#imports and config:
import re, regex, emoji
import pandas as pd
import numpy as np
import tqdm

import wandb
from datasets import load_dataset
from transformers import pipeline
import evaluate


# WANDB CONFIG
PROJECT = "mlip-lab4-slices-2026"
ENTITY = None
RUN_NAME = "baseline_vs_candidate"


In [6]:
# Models to compare
MODELS = {
    "baseline_model": "cardiffnlp/twitter-roberta-base-sentiment-latest",
    "candidate_model":    "LYTinn/finetuning-sentiment-model-tweet-gpt2",
}

In [7]:
# Label normalization for tweet_eval (0/1/2 -> string labels)
ID2LABEL = {0: "negative", 1: "neutral", 2: "positive"}

# Many HF sentiment models output labels like LABEL_0 / LABEL_1 / LABEL_2
HF_LABEL_MAP = {"LABEL_0": "negative", "LABEL_1": "neutral", "LABEL_2": "positive"}

USE_HF_DATASET = True  # set False to use tweets.csv fallback

### Step 2 - Load a dataset from Hugging Face

In [8]:
if USE_HF_DATASET:
    ds = load_dataset("cardiffnlp/tweet_eval", "sentiment")
    df = pd.DataFrame(ds["test"]).head(500).copy()
    df["label"] = df["label"].map(ID2LABEL)
else:
    df = pd.read_csv("tweets.csv")
    # Ensure it has 'text' and 'label' columns
    df = df.rename(columns={c: c.strip() for c in df.columns})
    assert {"text","label"}.issubset(df.columns), "tweets.csv must include text,label"
    df["label"] = df["label"].astype(str).str.lower()

df = df[["text","label"]].dropna().reset_index(drop=True)
df.head(3)


,text,label
0,@user @user what do these '1/2 naked pics' hav...,neutral
1,OH: “I had a blue penis while I was this” [pla...,neutral
2,"@user @user That's coming, but I think the vic...",neutral


### Step 3 - Define Failure-Relevant Metadata

#TODO:
In this step, you will create **at least 5** metadata columns that help you slice and analyze model behavior in Weights & Biases (W&B).
These metadata columns should **capture meaningful properties of the data or model behavior that may influence performance**. You can define them using:

1. Value matching (e.g., tweets containing hashtags or mentions)
2. Regex patterns (e.g., negation words, strong sentiment terms like love or hate)
3. Heuristics (e.g., emoji count, all-caps text, tweet length buckets)

Each metadata column should correspond to a potential hypothesis about when or why a model might succeed or fail.
These columns will be propagated through inference and included in the final predictions_table logged to W&B.

After inference, your W&B table (df_long) will contain:
- The original tweet text
- Ground-truth sentiment labels
- Model predictions and confidence scores
- All metadata columns you defined for slicing

You will use these metadata fields in the W&B UI (via the ➕ Filter option) to:
- Create slices of the data
- Compare model behavior across slices
- Identify patterns, weaknesses, or regressions that are not visible in overall accuracy

In [12]:
# Step 3 – Add slicing metadata (text-only)

# Enhanced slicing with hypothesis-driven metadata
def count_emojis(text: str) -> int:
    return sum(ch in emoji.EMOJI_DATA for ch in str(text))

df["emoji_count"] = df["text"].apply(count_emojis).astype(int)
df["has_hashtag"] = df["text"].str.contains(r"#\w+", regex=True)
df["has_mention"] = df["text"].str.contains(r"@\w+", regex=True)
df["has_negation"] = df["text"].str.contains(r"\b(not|never|no)\b", regex=True)

# NEW METADATA
df["has_contrast"] = df["text"].str.contains(r"\b(but|however|although|yet)\b", case=False, regex=True)
df["many_exclamations"] = df["text"].str.contains(r"(\!{2,}|\!\?)", regex=True)
df["has_url"] = df["text"].str.contains(r"http\S+", regex=True)

df["length_bucket"] = pd.cut(
    df["text"].str.len(),
    bins=[0, 50, 100, 200, 1000, 10_000],
    labels=["0-50", "51-100", "101-200", "201-1000", "1001+"],
    include_lowest=True
).astype(str)

# Updated slice definitions
def get_slices(df_any: pd.DataFrame):
    return {
        "emoji_gt3": df_any["emoji_count"] > 3,
        "has_negation": df_any["has_negation"] == True,
        "has_hashtag": df_any["has_hashtag"] == True,
        "has_contrast": df_any["has_contrast"] == True,
        "many_exclamations": df_any["many_exclamations"] == True,
        "has_url": df_any["has_url"] == True,
        "has_mention": df_any["has_mention"] == True,
        "long_tweets": df_any["length_bucket"].astype(str).isin(["201-1000", "1001+"]),
    }


C:\Users\STUDENT\AppData\Local\Temp\ipykernel_26816\2516331889.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_negation"] = df["text"].str.contains(r"\b(not|never|no)\b", regex=True)
C:\Users\STUDENT\AppData\Local\Temp\ipykernel_26816\2516331889.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_contrast"] = df["text"].str.contains(r"\b(but|however|although|yet)\b", case=False, regex=True)
C:\Users\STUDENT\AppData\Local\Temp\ipykernel_26816\2516331889.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["many_exclamations"] = df["text"].str.contains(r"(\!{2,}|\!\?)", regex=True)


In [13]:
# Transformers requires a backend (PyTorch/TensorFlow/Flax). We'll use PyTorch.
try:
    import torch, transformers, sys
    print("torch:", torch.__version__)
    print("transformers:", transformers.__version__)
    print("CUDA available:", torch.cuda.is_available())
    print("Python:", sys.executable)
except Exception as e:
    raise RuntimeError("Install PyTorch before proceeding: pip install torch torchvision torchaudio") from e

torch: 2.6.0+cpu
transformers: 4.57.1
CUDA available: False
Python: c:\Users\STUDENT\anaconda3\python.exe


###  Step 4 – Run Inference (Two Models)

In this step, you'll use two HuggingFace sentiment analysis models to run inference on your dataset:

In [14]:
from tqdm.auto import tqdm

def run_pipeline(model_id: str, texts: list[str]):
    clf = pipeline(
        "text-classification",
        model=model_id,
        truncation=True,
        max_length=128,     # avoid truncation warnings
        framework="pt",
        device=-1           # CPU
    )
    # (Optional) sanity check label mapping for this model
    # print(model_id, clf.model.config.id2label)

    preds, confs = [], []
    for t in tqdm(texts, desc=f"Infer: {model_id}"):
        out = clf(t)[0]
        lbl = HF_LABEL_MAP.get(out["label"], out["label"])
        preds.append(lbl)
        confs.append(float(out["score"]))
    return preds, confs

pred_frames = []
texts = df["text"].tolist()

for model_name, model_id in MODELS.items():
    yhat, conf = run_pipeline(model_id, texts)
    tmp = df.copy()
    tmp["model"] = model_name
    tmp["pred"] = yhat
    tmp["conf"] = conf
    pred_frames.append(tmp)

df_long = pd.concat(pred_frames, ignore_index=True)

# Add a stable example id so reshaping won't silently drop duplicates
df_long["ex_id"] = df_long.groupby(["text", "label"]).ngroup()

df_long.head(5)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Infer: cardiffnlp/twitter-roberta-base-sentiment-latest:   0%|          | 0/500 [00:00<?, ?it/s]

Device set to use cpu


Infer: LYTinn/finetuning-sentiment-model-tweet-gpt2:   0%|          | 0/500 [00:00<?, ?it/s]

,text,label,emoji_count,has_hashtag,has_mention,has_negation,has_contrast,many_exclamations,has_url,length_bucket,model,pred,conf,ex_id
0,@user @user what do these '1/2 naked pics' hav...,neutral,0,False,True,True,False,False,False,51-100,baseline_model,negative,0.804726,113
1,OH: “I had a blue penis while I was this” [pla...,neutral,0,False,False,False,False,False,False,51-100,baseline_model,neutral,0.866949,363
2,"@user @user That's coming, but I think the vic...",neutral,0,False,True,False,True,False,False,51-100,baseline_model,neutral,0.763724,102
3,I think I may be finally in with the in crowd ...,positive,0,True,True,False,False,False,False,51-100,baseline_model,positive,0.774047,305
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",negative,0,False,True,False,False,False,False,101-200,baseline_model,neutral,0.416397,160


In [20]:
# Step 4.5 – Wide-format Table for Model Comparison (Optional but recommended)
# One row per tweet, with baseline + candidate predictions in columns
# TODO: Replace with your metadata
df_wide = df_long.pivot_table(
    index=[
        "ex_id", "text", "label",
        "emoji_count", "has_hashtag", "has_mention", "has_negation", "length_bucket", "has_contrast", "many_exclamations", "has_url"
    ],
    columns="model",
    values=["pred", "conf"],
    aggfunc="first"
).reset_index()

# Flatten column names (e.g., pred_baseline_model, conf_candidate_model)
df_wide.columns = ["_".join([c for c in col if c]).strip("_") for col in df_wide.columns]

df_wide.head(5)

,ex_id,text,label,emoji_count,has_hashtag,has_mention,has_negation,length_bucket,has_contrast,many_exclamations,has_url,conf_baseline_model,conf_candidate_model,pred_baseline_model,pred_candidate_model
0,0,"""Fatty Kim The Third"" 😭😭😭",neutral,3,False,False,False,0-50,False,False,False,0.486252,0.978987,neutral,neutral
1,1,"""Focusing on [alt rightists'] respectability.....",neutral,0,False,False,False,51-100,False,False,False,0.573571,0.999728,negative,neutral
2,2,"""Kim Fatty the Third""",negative,0,False,False,False,0-50,False,False,False,0.849732,0.937710,neutral,neutral
3,3,"""We have lost everything"": Syrians return to r...",neutral,0,True,True,False,51-100,False,False,False,0.751955,0.994244,negative,positive
4,4,"""who's the most wiped out white boy? Zac Efron...",neutral,0,False,False,False,51-100,False,False,False,0.561232,0.906539,neutral,positive


### Step 5: Compute Metrics (Accuracy + Slice Accuracy + Regression)

In [21]:
# TODO: Edit to work for your slices

#compute metrics model-wise
from sklearn.metrics import accuracy_score

def compute_accuracy(y_true, y_pred):
    return accuracy_score(list(y_true), list(y_pred))

# Overall accuracy by model (df_long: one row per (tweet, model))
overall = df_long.groupby("model").apply(
    lambda g: compute_accuracy(g["label"], g["pred"]),
    include_groups=False
)

# Slice accuracy table (uses df_long masks)
slice_table = wandb.Table(columns=["slice", "model", "accuracy"])
slice_metrics = {}

for slice_name, mask in get_slices(df_long).items():
    slice_metrics[slice_name] = {}
    for model_name, g in df_long[mask].groupby("model"):
        acc = float(compute_accuracy(g["label"], g["pred"]))
        slice_table.add_data(slice_name, model_name, acc)
        slice_metrics[slice_name][model_name] = acc

In [22]:
# TODO: Edit to work for your slices


# Regression-aware evaluation (df_eval: one row per tweet, both model outputs) 
# A regression is when the candidate gets something wrong that the baseline got right.
BASELINE = "baseline_model"
CANDIDATE = "candidate_model"

# Ensure ex_id exists (safe even if it already exists)
df_long = df_long.copy()
if "ex_id" not in df_long.columns:
    df_long["ex_id"] = df_long.groupby(["text", "label"]).ngroup()

# Build df_eval with metadata carried through
df_eval = (
    df_long.pivot_table(
        index=[
            "ex_id", "text", "label",
            "emoji_count", "has_hashtag", "has_mention", "has_negation", "length_bucket", "has_contrast", "many_exclamations", "has_url"
        ],
        columns="model",
        values=["pred", "conf"],
        aggfunc="first"
    )
    .reset_index()
)

# Flatten column names (pred_baseline_model, conf_candidate_model, etc.)
df_eval.columns = ["_".join([c for c in col if c]).strip("_") for col in df_eval.columns]

# Correctness flags
df_eval["baseline_correct"]  = df_eval[f"pred_{BASELINE}"] == df_eval["label"]
df_eval["candidate_correct"] = df_eval[f"pred_{CANDIDATE}"] == df_eval["label"]

# Regression / improvement flags
df_eval["regressed"]   = df_eval["baseline_correct"] & ~df_eval["candidate_correct"]
df_eval["improved"]    = ~df_eval["baseline_correct"] & df_eval["candidate_correct"]
df_eval["both_wrong"]  = ~df_eval["baseline_correct"] & ~df_eval["candidate_correct"]
df_eval["both_correct"]= df_eval["baseline_correct"] & df_eval["candidate_correct"]

# Confidence-conditional regression (candidate is confident AND worse than baseline)
df_eval["confident_regression"] = df_eval["regressed"] & (df_eval[f"conf_{CANDIDATE}"] >= 0.8)

# Global regression metrics
regression_rate = float(df_eval["regressed"].mean())
improvement_rate = float(df_eval["improved"].mean())
conf_reg_rate = float(df_eval["confident_regression"].mean())

print("Regression rate:", regression_rate)
print("Improvement rate:", improvement_rate)
print("Confident regression rate:", conf_reg_rate)

Regression rate: 0.408
Improvement rate: 0.108
Confident regression rate: 0.382


In [23]:
# TODO: Edit to work with your slices

# Define slices on df_eval (must use columns that exist in df_eval)
def get_slices_eval(df_any):
    return {
        "emoji_gt3": df_any["emoji_count"] > 3,
        "has_negation": df_any["has_negation"] == True,
        "has_hashtag": df_any["has_hashtag"] == True,
        "has_contrast": df_any["has_contrast"] == True,
        "many_exclamations": df_any["many_exclamations"] == True,
        "has_url": df_any["has_url"] == True,
        "has_mention": df_any["has_mention"] == True,
        "long_tweets": df_any["length_bucket"].astype(str).isin(["201-1000", "1001+"]),
    }

# Slice-level regression metrics table
reg_table = wandb.Table(columns=["slice", "metric", "value"])
reg_metrics = {}

for slice_name, mask in get_slices_eval(df_eval).items():
    g = df_eval[mask]
    if len(g) == 0:
        continue

    reg = float(g["regressed"].mean())
    imp = float(g["improved"].mean())
    conf_reg = float(g["confident_regression"].mean())

    reg_table.add_data(slice_name, "regression_rate", reg)
    reg_table.add_data(slice_name, "improvement_rate", imp)
    reg_table.add_data(slice_name, "confident_regression_rate", conf_reg)

    reg_metrics[slice_name] = {
        "regression_rate": reg,
        "improvement_rate": imp,
        "conf_reg_rate": conf_reg
    }



# Step 6 — #TODO: Log to W&B & Analyse Slices
# (Make sure PROJECT/ENTITY/RUN_NAME exist from Step 1)

In [24]:
# Step 6: Log to W&B

PROJECT = "mlip-lab4-slices-2026"
ENTITY = None
RUN_NAME = "baseline_vs_candidate_enhanced"
# Ensure we have a fresh run if re-running
if wandb.run is not None:
    wandb.finish()
    
run = wandb.init(project=PROJECT, entity=ENTITY, name=RUN_NAME)

# Log tables and metrics
wandb.log({"predictions_table": wandb.Table(dataframe=df_long)})
wandb.log({"slice_metrics": slice_table})
wandb.log({"regression_metrics": reg_table})
wandb.log({"df_eval": wandb.Table(dataframe=df_eval)})

# Log summary metrics
for model_name, acc in overall.items():
    wandb.summary[f"{model_name}_accuracy"] = float(acc)

wandb.summary["regression_rate"] = regression_rate
wandb.summary["improvement_rate"] = improvement_rate
wandb.summary["confident_regression_rate"] = conf_reg_rate

print("W&B run URL:", run.get_url())
run.finish()


c:\Users\STUDENT\anaconda3\Lib\site-packages\pydantic\main.py:308: UserWarning: Pydantic serializer warnings:
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  Expected `list[str]` but got `tuple` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


W&B run URL: https://wandb.ai/sddaiddo-carnegie-mellon-university/mlip-lab4-slices-2026/runs/olst1vhf


baseline_model_accuracy,0.698
candidate_model_accuracy,0.398
confident_regression_rate,0.382
improvement_rate,0.108
regression_rate,0.408


### Instructions: Exploring Slice-Based Evaluation in W&B

# Purpose
In this lab, you are evaluating a candidate sentiment model to decide whether it should replace an existing baseline (production) model.
You have already:
  - run both models on the same dataset
  - logged predictions, confidence scores, and metadata to W&B
  - created metadata that allows you to slice the data
The most important goal is to understand when and why models behave differently.
Overall accuracy alone is often misleading.

# What to do in W&B
1. Open your W&B run
  - Click the project link and open the latest run.
2. Explore the predictions table
  - Go to the Tables tab and open predictions_table.
  - Each row is one tweet × one model.
3. Create and analyze slices (most important)
  - Use filters to create meaningful slices 
    (e.g., negation, emojis, hashtags, long tweets).
  - For each slice:
    - Compare baseline vs candidate performance.
    - Compare slice accuracy to overall accuracy.
    - Inspect a few misclassified examples to identify patterns.
4. Visualize slice performance
  - Open slice_metrics.
  - Create bar charts comparing baseline vs candidate accuracy for at least two slices.
5. Discuss your findings with the TA
  - Explain why slicing reveals issues that overall accuracy hides.
  - Say whether the candidate model should be deployed and why.


In [31]:
# Saved slice notes
saved_slice_notes = [
    "Mentions: Drastic drop in accuracy (27% vs 70%) suggests candidate struggles with conversational context.",
    "Negation: Candidate model performs poorly (36% vs 57%), likely missing the scope of negation.",
    "Contrast: Candidate model handles sentiment flips worse than baseline (52% vs 72%).",
    "Emojis: Candidate model has 100% accuracy vs 0% baseline, suggesting potential over-reliance on surface features.",
    "URLs: Candidate model performs slightly better (56% vs 50%), showing robustness to external links."
]
pd.DataFrame(saved_slice_notes, columns=["Insights"])


,Insights
0,Mentions: Drastic drop in accuracy (27% vs 70%...
1,Negation: Candidate model performs poorly (36%...
2,Contrast: Candidate model handles sentiment fl...
3,Emojis: Candidate model has 100% accuracy vs 0...
4,URLs: Candidate model performs slightly better...


### Step 7 - Targeted stress testing with LLMs

TODO: 
In this step, you will use a Large Language Model (LLM) to generate test cases that specifically target a weakness you observed during slicing.

What to do:
1. Choose one slice where you noticed poor performance, regressions, or surprising behavior.
2. Write a short hypothesis (1–2 sentences) explaining why the model might struggle on this slice. Example:
“The model struggles with tweets that use slang and sarcasm.”
3. Use an LLM to generate 10 test cases designed to test this hypothesis.
These can include:
    - subtle or ambiguous cases
    - difficult or adversarial cases
    - small wording changes that affect sentiment
4. Re-run both models on the generated test cases (helper script given below.)
5. Briefly describe what you observed to the TA:
    - Did the same failures appear again?
    - notice any new failure patterns?
    - would this affect your confidence in deploying the model?

Your input can be in the following format:

> Examples:
> - @user @user That’s coming, but I think the victims are going to be Medicaid recipients.
> - I think I may be finally in with the in crowd #mannequinchallenge  #grads2014 @user
> 
> Generate more tweets using slangs.

Use our provided GPTs to start the task: [llm-based-test-case-generator](https://chatgpt.com/g/g-982cylVn2-llm-based-test-case-generator). If you do not have access to GPTs, use the plain ChatGPT or other LLM providers you have access to instead.

In [33]:
# TODO: Paste your 10 generated tweets here:
generated_slice_description = ["Hypothesis: The candidate model struggles with negation"]

generated_cases = [
    "I am not happy with this product.",
    "The result is not good at all.",
    "There are no issues with the service.",
    "I never liked this place.",
    "Not bad, actually quite decent.",
    "I don't hate it, but it's okay.",
    "It wasn't terrible, surprisingly.",
    "I am not impressed by the quality.",
    "Nothing went wrong during the process.",
    "This is no fun."
]


In [34]:
#Helper code to run models on synthetic test cases:

def run_on_generated_tests(texts, models=MODELS):
    rows = []
    for model_name, model_id in models.items():
        clf = pipeline(
            "text-classification",
            model=model_id,
            truncation=True,
            framework="pt",
            device=-1
        )
        for t in texts:
            out = clf(t)[0]
            rows.append({
                "text": t,
                "model": model_name,
                "pred": HF_LABEL_MAP.get(out["label"], out["label"]),
                "conf": float(out["score"])
            })
    return pd.DataFrame(rows)


In [35]:
generated_df = run_on_generated_tests(generated_cases)
generated_df

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Device set to use cpu


,text,model,pred,conf
0,I am not happy with this product.,baseline_model,negative,0.936887
1,The result is not good at all.,baseline_model,negative,0.920545
2,There are no issues with the service.,baseline_model,neutral,0.714066
3,I never liked this place.,baseline_model,negative,0.931402
4,"Not bad, actually quite decent.",baseline_model,positive,0.867502
5,"I don't hate it, but it's okay.",baseline_model,neutral,0.451993
6,"It wasn't terrible, surprisingly.",baseline_model,positive,0.746955
7,I am not impressed by the quality.,baseline_model,negative,0.905853
8,Nothing went wrong during the process.,baseline_model,neutral,0.722355
9,This is no fun.,baseline_model,negative,0.910570


In [30]:
# OPTIONAL: Log synthetic test cases to W&B
wandb.log({
    "synthetic_tests": wandb.Table(dataframe=generated_df)
})

Error: You must call wandb.init() before wandb.log()